# Immune Data Preprocessing

In [1]:
import anndata as ad
import pandas as pd
import scanpy as sc

In [3]:
## Set data folder
data_path = "../data/cellxgene_data"

In [4]:
## Load CZI immune data

adata1 = ad.read_h5ad(f"{data_path}/immune_cells_chunk0.h5ad") # chunk 1
adata2 = ad.read_h5ad(f"{data_path}/immune_cells_chunk1.h5ad") # chunk 2

# Axis 0 = cells
adata_full = ad.concat([adata1, adata2], axis=0, join="outer", merge="same")
for field in adata_full.obs.keys().tolist():
    if pd.api.types.is_categorical_dtype(adata_full.obs[field]):
        adata_full.obs[field] = adata_full.obs[field].cat.remove_unused_categories() # remove unused categories
adata_full.write_h5ad(f"{data_path}/immune_1M_merged.h5ad") # write to single .h5ad

/home/s5srinivasan/immunegptenv/lib64/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_403002/3272363573.py:9: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(adata_full.obs[field]):


## Inspection

In [5]:
## Inspect

adata_full.obs

,soma_joinid,dataset_id,assay,cell_type,cell_type_ontology_term_id,development_stage,development_stage_ontology_term_id,disease,disease_ontology_term_id,donor_id,observation_joinid,self_reported_ethnicity,self_reported_ethnicity_ontology_term_id,sex,sex_ontology_term_id,tissue_general,tissue_general_ontology_term_id
0,4276120,d18736c3-6292-4379-919a-d6d973204c87,10x 3' v3,"effector memory CD4-positive, alpha-beta T cell",CL:0000905,young adult stage,HsapDv:0000266,normal,PATO:0000461,Control10_1_C_F,WE&Q3GE(19,European,HANCESTRO:0005,female,PATO:0000383,blood,UBERON:0000178
1,4276173,d18736c3-6292-4379-919a-d6d973204c87,10x 3' v3,"central memory CD4-positive, alpha-beta T cell",CL:0000904,young adult stage,HsapDv:0000266,normal,PATO:0000461,Control10_1_C_F,D(*#;_I22y,European,HANCESTRO:0005,female,PATO:0000383,blood,UBERON:0000178
2,4276224,d18736c3-6292-4379-919a-d6d973204c87,10x 3' v3,gamma-delta T cell,CL:0000798,adult stage,HsapDv:0000258,normal,PATO:0000461,Control15_1_C_F,%KsHiyf)`1,European,HANCESTRO:0005,female,PATO:0000383,blood,UBERON:0000178
3,4276469,d18736c3-6292-4379-919a-d6d973204c87,10x 3' v3,"central memory CD4-positive, alpha-beta T cell",CL:0000904,60-79 year-old stage,HsapDv:0000272,normal,PATO:0000461,Control07_1_C_F,z~n6JxbiyO,European,HANCESTRO:0005,female,PATO:0000383,blood,UBERON:0000178
4,4276566,d18736c3-6292-4379-919a-d6d973204c87,10x 3' v3,classical monocyte,CL:0000860,60-79 year-old stage,HsapDv:0000272,normal,PATO:0000461,Control07_1_C_F,>{%;4*}I|S,European,HANCESTRO:0005,female,PATO:0000383,blood,UBERON:0000178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,106479716,53d208b0-2cfd-4366-9866-c3c6114081bc,10x 3' v3,pericyte,CL:0000669,60-year-old stage,HsapDv:0000154,normal,PATO:0000461,TSP25,DHu5I+@`x_,Asian,HANCESTRO:0008,male,PATO:0000384,lung,UBERON:0002048
499996,106479735,53d208b0-2cfd-4366-9866-c3c6114081bc,10x 3' v3,basophil,CL:0000767,60-year-old stage,HsapDv:0000154,normal,PATO:0000461,TSP25,A%cQ~Y$sCM,Asian,HANCESTRO:0008,male,PATO:0000384,lung,UBERON:0002048
499997,106479739,53d208b0-2cfd-4366-9866-c3c6114081bc,10x 3' v3,club cell,CL:0000158,60-year-old stage,HsapDv:0000154,normal,PATO:0000461,TSP25,9^j1cWgF3%,Asian,HANCESTRO:0008,male,PATO:0000384,lung,UBERON:0002048
499998,106479761,53d208b0-2cfd-4366-9866-c3c6114081bc,10x 3' v3,"CD4-positive, alpha-beta T cell",CL:0000624,60-year-old stage,HsapDv:0000154,normal,PATO:0000461,TSP25,W5>{{cBDJk,Asian,HANCESTRO:0008,male,PATO:0000384,lung,UBERON:0002048


In [3]:
# Assert all metadata fields

ontology_metadata_fields = [
                    # "cell_type_ontology_term_id",
                    "self_reported_ethnicity_ontology_term_id", 
                    # "tissue_general_ontology_term_id",
                    # "development_stage_ontology_term_id"
                    # "disease" # use DOID:4 - ID for root
]

# Get all unique ontology IDs per category/field
ontology_metadata_ids = {
    ontology: sorted(adata_full.obs[ontology].dropna().unique().tolist()) # same result with or without dropna(), which means all entries are annotated
    for ontology in ontology_metadata_fields
}

# Get all unique ontology IDs per category/field
cell_counts_per_category = {
    ontology: adata_full.obs[ontology].value_counts().to_dict()
    for ontology in ontology_metadata_fields
}

NameError: name 'adata_full' is not defined

In [1]:
for key in ontology_metadata_ids:
    print(key,len(ontology_metadata_ids[key]))

NameError: name 'ontology_metadata_ids' is not defined

In [8]:
cell_counts_per_category

{'cell_type_ontology_term_id': {'CL:0000236': 92735,
  'CL:0000624': 78053,
  'CL:0000895': 63227,
  'CL:0000625': 44281,
  'CL:0000623': 44211,
  'CL:0000583': 36360,
  'CL:0001054': 26980,
  'CL:0000904': 26297,
  'CL:0002063': 22761,
  'CL:0000860': 22561,
  'CL:0000787': 22355,
  'CL:0000775': 21275,
  'CL:0000837': 18456,
  'CL:0000905': 17389,
  'CL:0000900': 17036,
  'CL:0000843': 16973,
  'CL:0002062': 16900,
  'CL:0000576': 16486,
  'CL:0000939': 16088,
  'CL:0000913': 15903,
  'CL:0002038': 14292,
  'CL:0000788': 13110,
  'CL:0000815': 13097,
  'CL:0000038': 12950,
  'CL:0000786': 12837,
  'CL:0000557': 12819,
  'CL:0008001': 12223,
  'CL:0000235': 11499,
  'CL:0000084': 9880,
  'CL:0000037': 8012,
  'CL:0000810': 7879,
  'unknown': 7290,
  'CL:0000938': 6271,
  'CL:0001062': 6178,
  'CL:0000049': 5985,
  'CL:0001203': 5920,
  'CL:0002138': 5505,
  'CL:0002144': 5394,
  'CL:0001058': 5309,
  'CL:0000798': 5243,
  'CL:1001603': 5146,
  'CL:0000115': 5143,
  'CL:0000906': 4994,